### Introduction

Understanding how neurons fire – and being able to predict their activity – is crucial for neuroscience and brain-inspired technology. Traditional neuron models like the Hodgkin-Huxley (HH) equations give deep biological insight by modeling ion channels and membrane voltages, but they are complex and computationally heavy​. Simpler models such as the integrate-and-fire neuron capture the idea of a neuron spiking when a threshold is reached, but they oversimplify many details​. Modern machine learning (ML) offers a different approach: we can train artificial neural networks (ANNs) directly on neural data to mimic neuronal behavior. These ML models can learn patterns from large, high-dimensional datasets efficiently​, but they operate as “black boxes,” offering less insight into the biological mechanisms​ . This project aims to compare these approaches – the mechanistic models versus learned ML models – using real neuronal firing data. In simple terms, we want to see if a straightforward ANN can behave like a neuron (predicting when it will spike) and how that compares to classic biology-based models. The significance of this comparison is to bridge the gap between interpretability and predictive power​. If successful, we gain a modeling approach that is both accurate and somewhat biologically informed, helping us understand brain function and design better neurotechnology. The long-term vision is to achieve a balance where we don’t sacrifice scientific insight for predictive accuracy (or vice versa), thus bringing together neuroscience and AI in a practical way.

### Workflow

Our workflow follows a typical data-driven modeling pipeline. First, we start with raw electrophysiological data – in this case, intracellular voltage recordings from individual neurons in response to injected current stimuli. These raw data are then preprocessed to clean and format them (for example, filtering out noise or normalizing signals). Next comes feature extraction, where we derive informative measures from the raw signals, such as spike times, firing rates, inter-spike intervals (ISIs), or spike waveform shapes. With these features (or the processed signals themselves), we then train an ML model (an artificial neural network) to predict neuronal responses. Finally, we evaluate the model’s performance by comparing its outputs to the actual neural data, using metrics like spike timing accuracy, firing rate error, etc. The flowchart above illustrates these steps: from raw data to preprocessing, feature extraction, modeling, and evaluation. This structured approach ensures we can systematically assess how well the ML model captures neural behavior at each stage.

### Background

Traditional models like HH are on one end of the spectrum – “very realistic but heavy” – whereas simple models like integrate-and-fire are “lightweight but oversimplified.” The HH model was a breakthrough that described how action potentials (spikes) are initiated and propagated using ion channel dynamics (for which Hodgkin and Huxley won the Nobel Prize)​. It remains one of the most successful and widely used models in computational neuroscience, but it involves solving four coupled differential equations and many parameters per neuron, which is impractical for large networks. In contrast, the integrate-and-fire model (first proposed by Louis Lapicque in 1907) treats the neuron as a basic integrator of input current until a threshold is reached, then emits a spike​. This simplicity allows simulating thousands of neurons in real time, but the model doesn’t explain how a spike occurs biologically – it just says “if input > θ, then spike.” 

ML models (ANNs), on the other hand, forego explicit biophysics entirely. They can be trained to take a neuron’s input (say, the injected current or preceding voltage values) and directly predict the neuron’s output (spike or no spike, or the membrane voltage at the next time step). The upside is that ANNs can potentially learn any input–output mapping given enough data – even complex nonlinear neuron behaviors – without us hand-crafting equations. In fact, researchers have shown that a simple ANN can approximate the spike timing of detailed models quite well if trained properly​. The downside is that we lose the built-in interpretability: unlike HH, which tells us about ion channels, an ANN is essentially a fitted function with many weights that don’t correspond to physical quantities. This trade-off between biological realism and predictive power is at the heart of our project’s motivation​. 

Notably, there are also “middle-ground” models. For example, the Adaptive Exponential Integrate-and-Fire (AdEx) model introduced by Brette and Gerstner adds a couple of extra parameters to the basic integrate-and-fire to mimic spike frequency adaptation, capturing a wider range of neuron behaviors without full HH complexity​. Likewise, Eugene Izhikevich proposed a simple 2-variable spiking model that is as computationally cheap as integrate-and-fire yet can produce many realistic firing patterns by tuning just a few parameters. These attempts highlight the ongoing effort to balance complexity and simplicity. Our project, however, takes a different tack by exploring ML – effectively letting the computer “learn” the neuron’s input-output function from data, rather than relying on biophysical assumptions.

### Dataset

We will use the Allen Institute’s Brain Atlas Cell Types Database as our data source. This is a rich open dataset of electrophysiological recordings from individual neurons, collected via whole-cell patch clamp experiments​. In particular, the Cell Types Database provides for each neuron:

Intracellular voltage recordings in response to various current stimuli (current injection sweeps), i.e. the raw voltage vs time traces where we see spikes.

Metadata about each cell (species, brain region, Cre line, etc.), and sometimes morphological reconstructions and transcriptomic info, though our focus is on the electrophysiology.

Pre-computed features for each cell’s activity (e.g. resting potential, input resistance, spike threshold, etc.) which we can use for validation or as reference​.

The electrophysiology data are stored in Neurodata Without Borders (NWB) format, which is an HDF5-based standard for neurophysiology data​. Essentially, each cell’s recording is in an .nwb file (which is a specialized HDF5 file) that contains the time series of stimuli and responses, and sometimes identified spike times. We can access these data in Python in multiple ways: the Allen Institute provides the AllenSDK (Software Development Kit) with a convenient API, and one can also use generic HDF5 tools or the pynwb library. We’ll use the AllenSDK for ease of use. For example, the AllenSDK’s CellTypesCache class can automatically download the NWB file for a given cell ID and open it for us​. This saves us from dealing with low-level file details. Under the hood, CellTypesCache.get_ephys_data(specimen_id) will fetch the NWB and return an NwbDataSet object, from which we can retrieve the recorded stimulus and response traces for each experimental sweep (trial)​

- Resources and Repositories: In preparing to use this dataset, the following resources are very useful:
AllenSDK Documentation and Examples: The AllenSDK docs include example Jupyter notebooks on how to download stimuli and responses, plot them, and even compute features​. We leveraged these examples heavily to get started.
- Allen Cell Types API: The Allen Brain Map API can be used to query cells and download data programmatically. The AllenSDK is basically a wrapper around this.
- Neurodata Without Borders (NWB): Understanding NWB structure helps if we ever need to go beyond the SDK. NWB is an initiative to standardize neuro data formats​, meaning our data isn’t in some idiosyncratic format – it’s in a widely supported schema (which is great for reproducibility).
- GitHub Repositories: There are community-supported tools to interface with this database. For example, the stripathy/AIBS_cell_types repository provides example notebooks for accessing the Allen Cell Types data and even cross-referencing it with NeuroElectro​. Another is the scidash/neuronunit project, which includes code (aibs.py) to fetch data from the Allen database for model validation purposes​. The Allen Institute’s own GitHub hosts an IPFX (Intracellular Physiology Feature Extraction) library for calculating spike features – essentially the same code they used internally to generate the database features​. We’ll make use of such tools rather than reinvent the wheel.
  
In summary, the Allen Cell Types Database gives us a high-quality, curated source of ground truth neuronal behavior. It also offers an opportunity to focus on a specific brain region. The dataset includes cells from mouse visual cortex (V1) and a few other areas; for clarity we may restrict to V1 neurons, since those have been well-characterized (e.g. the landmark study by Gouwens et al. clustered over 1000 V1 neurons into types based on these same recordings​
). Concentrating on one region (say, V1) means the neurons we model have some common functional context, which simplifies analysis and interpretation.

### <center> Related Work/ References 

1. `Hodgkin & Huxley (1952) A Quantitative Description of Membrane Current… (J. Physiol.)`​ 

The classic paper that introduced the ion-channel model of the action potential. This set the foundation for all biophysical neural modeling with its extremely accurate (but complex) description of spike generation.

2. `Eugene Izhikevich (2003) – “Simple Model of Spiking Neurons” (IEEE TNN)`

Proposed a simple four-parameter neuron model that can produce a wide variety of spiking patterns. It’s as biologically plausible as HH in behavior, but as efficient as integrate-and-fire. Illustrates the compromise between realism and simplicity clearly.
    
3. `Brette & Gerstner (2005) – Adaptive Exponential Integrate-and-Fire model (J. Neurophysiol.)`

Developed the AdEx model​, which adds adaptation to integrate-and-fire. This model became a standard because it can closely approximate the spike trains of cortical neurons while remaining computationally light. It’s a great example of enhancing interpretability (parameters like “tau” and “threshold” still meaningful) while improving predictive power​

4. `Jolivet et al. (2008) – Benchmark Test for Simple Neuron Models (J. Neurosci. Methods)`

Conducted quantitative tests on how well simple neuron models can predict the spike timing of a detailed model or real neuron. Notably showed that with proper fitting, even integrate-and-fire variants can predict spike times with surprising accuracy​ (within a few milliseconds for many spikes), reinforcing that simplified models can be tuned to be predictive.

5. `Teeter et al. (2018) – Generalized LIF models classify multiple neuron types (Nat. Comm.)​`

The Allen Institute team fit a series of GLIF models to hundreds of neurons from the Allen Cell Types database. They demonstrated that increasing model complexity (adding adaptation, etc.) improved spike prediction on held-out stimuli and that the fitted model parameters could be used to classify neuron types. This is directly relevant to our project as it uses the same data and shows how far an interpretable model can go in predicting real neuron behavior.
 
6. `Gouwens et al. (2019) – Classification of electrophysiological and morphological neuron types in the mouse visual cortex (Nat. Neurosci.)​`

A massive data-driven study (from the Allen Institute) that used techniques like PCA and clustering on electrophysiological features to define neuron types. While not an “ML model for dynamics,” it highlights the power of data-driven approaches in neuroscience and provides context that our data (V1 neurons) indeed have consistent patterns that models can exploit.
   
7. `Wang et al. (2022) – “Predicting Spike Features of HH-type Neurons with ANN” (Front. Comp. Neurosci.)​`

This recent paper explicitly bridged HH models and ANN: they trained ANNs to predict the occurrence and properties of spikes (max voltage, etc.) in various HH-model neurons. They achieved high accuracy and much faster simulation, suggesting ANN “surrogates” for biophysical models can maintain fidelity​. This directly inspires our approach of using ANNs to mimic neuronal firing.
     
8. `Oláh et al. (2022) – Accelerating biophysical simulations with ML (eLife)​`

Demonstrated a method to replace expensive network simulations of detailed neuron models with ANNs, speeding up simulation by orders of magnitude. It’s an example of using ML to achieve predictive power (simulate many neurons quickly) while trying to preserve biological realism (the ANN was trained on data from detailed models, so it inherits their realism). This shows the feasibility of our idea on a larger scale.

9. `Yamins et al. (2014) – Performance-optimized models of visual cortex (PLOS Comp. Bio / Science)​`

A seminal work where deep convolutional neural networks (trained on object recognition tasks) were shown to predict the responses of neurons in primate visual cortex (IT) better than previous models. This is a different scale (population and higher-level), but it’s a key example of ML models (CNNs) reaching neuroscience prediction accuracy that rivals the brain’s own representational patterns. It highlights the theme that models optimized purely for a task can end up very predictive of neural activity, even if they are not constrained to be biologically realistic. That informs the philosophy that sometimes, predictive power alone can reveal aspects of brain function.
   
10. `Richards et al. (2019) – “A deep learning framework for neuroscience” (Nat. Neurosci.)`

A perspective article that argues for integrating deep learning models into neuroscience research. It discusses how biologically inspired constraints can be added to ANN models and how such models can be used as hypotheses for brain function. This broad context motivates projects like ours, where we use deep learning not just as a engineering tool but as a scientific modeling tool, while being aware of the interpretability issues.